
# Prerequisites – Download nltk stopwords and spacy model

In [1]:
# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/darora/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Import Packages


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
# spacy for lemmatization
import spacy

In [5]:

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

/Users/darora/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/Users/darora/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Prepare Stopwords

In [8]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

sw = pd.read_csv('VP_assertions_stop_words_5303_W2021.txt')

# Convert to list
sw_data = sw.Archive.tolist()

stop_words.extend(sw_data)


# Import Assrtions Data

In [9]:
df = pd.read_csv('230 VP assertions corpus - Group 6.csv')

In [10]:
df.head(5)

,name,content
0,G6A002,Access resources required to scale at relative...
1,A003,Adapt offers to each market
2,A005,"Align interests of investors, the company top ..."
3,A006,Allow resource owners to make money using your...
4,A007,Apply big data analytics to produce insightful...


# Data pre-processing

In [11]:
# Convert to list
data = df.content.values.tolist()

In [12]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

In [13]:
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [14]:
pprint(data[:1])

['Access resources required to scale at relatively low cost or for free by '
 'creating benefits for the resource owners that they cannot create alone']


# Tokenize words and Clean-up text



In [15]:
#Tokenize each sentence into a list of words, removing punctuations and unnecessary characters 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['access', 'resources', 'required', 'to', 'scale', 'at', 'relatively', 'low', 'cost', 'or', 'for', 'free', 'by', 'creating', 'benefits', 'for', 'the', 'resource', 'owners', 'that', 'they', 'cannot', 'create', 'alone']]


# Creating Bigram and Trigram Models

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  


bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['access', 'resources', 'required', 'to', 'scale', 'at', 'relatively', 'low', 'cost', 'or', 'for', 'free', 'by', 'creating', 'benefits', 'for', 'the', 'resource_owners', 'that', 'they', 'cannot', 'create', 'alone']


# Remove Stopwords & Make Bigrams

In [17]:
# Define functions for stopwords, 
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [18]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [19]:
pprint(data_words_bigrams[:1])

[['resources', 'low', 'benefits', 'resource_owners']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

In [21]:
# Create Corpus
texts = data_words_bigrams

In [22]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [23]:

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]


In [24]:
id2word[0]

'benefits'

In [25]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('benefits', 1), ('low', 1), ('resource_owners', 1), ('resources', 1)]]

# Building the Topic Model

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# View the topics in LDA model

In [27]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.003*"resources" + 0.003*"existing" + 0.003*"innovations" + '
  '0.003*"treatment" + 0.003*"allocate" + 0.003*"customizing" + 0.003*"list" + '
  '0.003*"wishes" + 0.003*"preferentially" + 0.003*"advocacy"'),
 (1,
  '0.618*"stakeholders" + 0.001*"strategic" + 0.001*"budget" + '
  '0.001*"agencies" + 0.001*"government" + 0.001*"limitations" + '
  '0.001*"preferred" + 0.001*"reconcile" + 0.001*"aligned" + '
  '0.001*"allocating"'),
 (2,
  '0.156*"products" + 0.133*"benefit" + 0.112*"chain" + 0.089*"services" + '
  '0.049*"brand" + 0.045*"ways" + 0.041*"channels" + 0.023*"barriers" + '
  '0.010*"interfaces" + 0.010*"applications"'),
 (3,
  '0.336*"product" + 0.115*"customers" + 0.088*"service" + 0.070*"brand" + '
  '0.057*"products" + 0.023*"benefits" + 0.009*"competitor" + 0.009*"terms" + '
  '0.009*"bound" + 0.001*"anchor"'),
 (4,
  '0.359*"proposition" + 0.082*"resource" + 0.078*"skills" + 0.066*"portfolio" '
  '+ 0.044*"investors" + 0.038*"returns" + 0.030*"board" + 0.009*"gov

# Visualize the topics-keywords

In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.388095  0.026941       1        1  20.330909
10     0.034413 -0.270421       2        1  14.182069
14     0.023973 -0.233331       3        1  10.736521
16    -0.051722  0.041911       4        1   6.016209
3      0.215916  0.067298       5        1   5.978373
12     0.005355 -0.022737       6        1   5.852607
2      0.120481  0.094531       7        1   5.618929
4     -0.043616 -0.130120       8        1   4.824397
1     -0.052176  0.041860       9        1   4.418427
19    -0.068168  0.047611      10        1   4.097920
17    -0.064022  0.048121      11        1   3.320983
8     -0.062691  0.052398      12        1   3.038519
15    -0.066120  0.055491      13        1   2.298502
13    -0.047450  0.051444      14        1   1.850640
6     -0.057288 -0.023487      15        1   1.813950
18    -0.058132 -0.011344      16        1   1.611260
11    -0.061319  0.044232      17        1   1.509612
5     -0.057619  0.045464      18        1   1.389159
7     -0.048955  0.037069      19        1   0.555507
0     -0.048955  0.037069      20        1   0.555507, topic_info=            Term       Freq      Total Category  logprob  loglift
10     customers  85.000000  85.000000  Default  30.0000  30.0000
30  stakeholders  29.000000  29.000000  Default  29.0000  29.0000
9       customer  42.000000  42.000000  Default  28.0000  28.0000
85     employees  29.000000  29.000000  Default  27.0000  27.0000
4         offers  26.000000  26.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
25     companies   0.016649   4.356067  Topic20  -5.8579  -0.3739
26        border   0.016649   4.036867  Topic20  -5.8579  -0.2978
27      investor   0.016649  10.870348  Topic20  -5.8579  -1.2884
28   stakeholder   0.016649   9.429638  Topic20  -5.8579  -1.1462
29          user   0.016649   2.011967  Topic20  -5.8579   0.3985

[853 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
92       14  0.435265  achievements
93       14  0.803869  affiliations
42        7  0.709241  applications
55        7  0.458823      barriers
88        7  0.923514       benefit
...     ...       ...           ...
29       18  0.497026          user
18        1  0.688389         users
18       14  0.172097         users
99        2  0.802652        vision
52        7  0.858721          ways

[112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 11, 15, 17, 4, 13, 3, 5, 2, 20, 18, 9, 16, 14, 7, 19, 12, 6, 8, 1])